## Run circuits on IBM Quantum devices in 2024

For games run using manual Qiskit, the circuits are saved in QASM. Format. This notebook can be used to run those files on modern IBM Quantum hardware, using the latest Qiskit version (as of July 2024).

In [1]:
from qiskit import QuantumCircuit, transpile

import sys
sys.path.append("../")
from devices import supportedDevices, getLayout
from QuantumAwesomeness import *

First, set up the provider. You made need to add login credentials.

In [2]:
from qiskit_ibm_provider import IBMProvider
provider = IBMProvider()

Then specify the device.

In [3]:
device = 'ibm_fez'
backend = provider.get_backend(device)

Next we retreive, transpile and run the circuits. The job IDs are output in a file.

In [ ]:
file_ids = open('../results/'+device+'/jobids_move=C_shots=8192_sim=False.txt', 'a')
for s in range(20):
    with open('../results/'+device+'/qasm/qasm_C_8192_'+str(s)+'.txt', 'r') as file:
        qasms = file.readlines()[1::]

    qcs = []
    for qasm in qasms:
        qc = QuantumCircuit()
        qc = qc.from_qasm_str(qasm.replace('\\n','\n'))
        tqc = transpile(qc, backend)
        qcs.append(tqc)
    #print(qc.count_ops(), tqc.count_ops())

    job = backend.run(qcs, shots=8192, rep_delay=backend.configuration().rep_delay_range[1])

    file_ids.write(job.job_id()+'\n')
    print(job.job_id())
file_ids.close()

Once the jobs have finished running, the results can be retreived.

In [4]:
with open('../results/'+device+'/jobids_move=C_shots=8192_sim=False.txt', 'r') as file:
    jobids = file.readlines()

results = []
for jobid in jobids:
    if jobid:
        jobid = jobid.replace('\n','')
        job = provider.retrieve_job(jobid)
        result = []
        for result_raw in job.result().get_counts():
            result.append({})
            for string, nums in result_raw.items():
                result[-1][string[::-1]] = nums
        results.append(result)

file = open('../results/'+device+'/results_move=C_shots=8192_sim=False.txt', 'a')
for result in results:
    file.write(str(result))
file.close()

They can then be processed to get and save the required information.

In [5]:
num, area, entangleType, pairs, pos, example, sdk, runs = getLayout(device)

file_one = open('../results/'+device+'/oneProbs_move=C_shots=8192_sim=False.txt', 'a')
file_same = open('../results/'+device+'/sameProbs_move=C_shots=8192_sim=False.txt', 'a')
for r, result in enumerate(results):
    oneProbs = []
    sameProbs = []
    for resultsRaw in result:
        oneProb, sameProb, _ = processResults ( dict(resultsRaw), num, pairs, False, 8192 )
        oneProbs.append(oneProb)
        sameProbs.append(sameProb)
    file_one.write((r!=0)*'\n'+str(oneProbs))
    file_same.write((r!=0)*'\n'+str(sameProbs))
file_one.close()
file_same.close()